In [1]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)
import torch
import torch.nn as nn
import numpy as np
import time
import os
import matplotlib.pyplot as plt
import torchvision.utils as vutils
from torchvision.utils import save_image

# Kiểm tra nếu có GPU khả dụng, nếu không thì sử dụng CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

True
12.4


device(type='cuda')

In [2]:
import torch

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Có {num_gpus} GPU(s) được phát hiện:")
    
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("Không có GPU nào được phát hiện.")


Có 1 GPU(s) được phát hiện:
GPU 0: Quadro RTX 4000 with Max-Q Design


In [3]:
import os
from PIL import Image
import torch
from torchvision import transforms

def load_and_shuffle_images(folder_path, mean, std, img_shape=(128, 128), device='cpu'):
    images = []
    
    # Định nghĩa các transform để xử lý ảnh
    transform = transforms.Compose([
        transforms.Resize(img_shape),  # Resize ảnh về kích thước (128, 128)
        transforms.ToTensor(),  # Chuyển đổi ảnh thành tensor và chuẩn hóa về [0, 1]
        transforms.Normalize(mean, std)  # Chuẩn hóa về [-1, 1]
    ])
    
    # Đọc và transform ảnh
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = Image.open(img_path).convert("RGB")  # Đảm bảo ảnh có 3 kênh (RGB)
        img_tensor = transform(img).to(device)  # Áp dụng các transform lên ảnh và chuyển tensor lên GPU
        images.append(img_tensor)
    
    # Shuffle các tensor ảnh
    images_tensor = torch.stack(images)  # Chuyển danh sách ảnh thành tensor với kích thước (N, 3, 128, 128)
    shuffled_indices = torch.randperm(images_tensor.size(0), device=device)  # Tạo thứ tự ngẫu nhiên để shuffle trên GPU
    shuffled_images_tensor = images_tensor[shuffled_indices]  # Trộn các tensor ảnh
    
    return shuffled_images_tensor

# Thiết lập mean và std để chuẩn hóa ảnh
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

# Thay đổi đường dẫn tới thư mục chứa ảnh 
folder_path = rf'D:\Download\Edge\stinkbug\Merge\Labeling\Stink_bugs\Categories\Images\Inputs\256\aug\adult'  
img_shape = (256, 256)

# Kiểm tra xem có GPU không và thiết lập thiết bị phù hợp
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load và shuffle ảnh trên GPU
real_images = load_and_shuffle_images(folder_path, mean, std, img_shape, device=device)

# Lưu dữ liệu tensor
tensor_path = rf'D:\Download\Edge\stinkbug\Merge\Labeling\Stink_bugs\Categories\torch\adult_aug_real_images_256.pt'
torch.save(real_images.cpu(), tensor_path)  # Lưu tensor về CPU để không gặp lỗi khi lưu

# Kiểm tra kích thước tensor
print(f"Loaded and shuffled {real_images.size(0)} images with shape {real_images.shape}")


Using device: cuda
Loaded and shuffled 7254 images with shape torch.Size([7254, 3, 256, 256])


In [4]:
tensor_path = rf'D:\Download\Edge\stinkbug\Merge\Labeling\Stink_bugs\Categories\torch\adult_aug_real_images_256.pt'
img_shape = (256, 256)
# Load dữ liệu tensor từ file
real_images = torch.load(tensor_path)

# Kiểm tra kích thước của dữ liệu đã load
print(f"Loaded {real_images.size(0)} images with shape {real_images.shape}")


C:\Users\Lappro\AppData\Local\Temp\ipykernel_35812\1651777266.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  real_images = torch.load(tensor_path)


Loaded 7254 images with shape torch.Size([7254, 3, 256, 256])


In [17]:
import torch
import torch.nn as nn

# Define the Generator
class Generator(nn.Module):
    def __init__(self, nz, ngf, nc):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),  # Fixed channels from ngf * 4 to ngf * 8
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)


# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self, nc, ndf):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)


In [18]:
def initialize_models(nz, ngf, ndf, nc, device):
    netG = Generator(nz, ngf, nc).to(device)
    netD = Discriminator(nc, ndf).to(device)
    return netG, netD


In [23]:
import os
import torch
import torch.optim as optim
import torchvision.utils as vutils
import matplotlib.pyplot as plt
from tqdm import tqdm  # Import tqdm

# Define the Generator and Discriminator classes (as shown earlier)

def train_dcgan_single_class_from_tensor(netG, netD, real_images, nz, num_epochs, lr, beta1, device, save_dir, batch_size):
    criterion = nn.BCELoss()
    optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
    optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

    G_losses = []
    D_losses = []
    real_images = real_images.to(device)  # Ensure real_images is on the correct device

    # Use tqdm to add a progress bar
    for epoch in tqdm(range(num_epochs), desc="Training Epochs"):
        for i in range(0, real_images.size(0), batch_size):
            # Get a batch of real images
            real_batch = real_images[i:i + batch_size]

            # Update Discriminator
            netD.zero_grad()

            output = netD(real_batch).view(-1)
            label = torch.full(output.size(), 1, dtype=torch.float, device=device)  # Label for real images

            lossD_real = criterion(output, label)
            lossD_real.backward()

            # Generate fake images and update Discriminator with fake images
            noise = torch.randn(real_batch.size(0), nz, 1, 1, device=device)
            fake_images = netG(noise)
            output = netD(fake_images.detach()).view(-1)
            label = torch.full(output.size(), 0, dtype=torch.float, device=device)  # Label for fake images

            lossD_fake = criterion(output, label)
            lossD_fake.backward()
            optimizerD.step()

            # Update Generator
            netG.zero_grad()
            label.fill_(1)  # Generator tries to trick Discriminator, so label fake images as real (1)
            output = netD(fake_images).view(-1)
            lossG = criterion(output, label)
            lossG.backward()
            optimizerG.step()

            # Save Losses
            G_losses.append(lossG.item())
            D_losses.append(lossD_real.item() + lossD_fake.item())

        # Save images every 5% of epochs
        if epoch % (num_epochs // 20) == 0:
            with torch.no_grad():
                fake_images = netG(noise).detach().cpu()
            img_save_path = os.path.join(save_dir, f'epoch_{epoch}_fake_samples.png')
            vutils.save_image(fake_images[:10], img_save_path, normalize=True)
            print(f'Saved generator output at epoch {epoch} to {img_save_path}')

        # Print losses every 10 epochs
        if epoch % 10 == 0:
            print(f'Epoch [{epoch}/{num_epochs}] Loss_D: {D_losses[-1]:.4f} Loss_G: {G_losses[-1]:.4f}')

    # Visualize Losses
    plt.figure(figsize=(10,5))
    plt.title("Generator and Discriminator Loss During Training")
    plt.plot(G_losses, label="G")
    plt.plot(D_losses, label="D")
    plt.xlabel("iterations")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

# Create folder to save results
save_dir = "gan_training_results_single_class"
os.makedirs(save_dir, exist_ok=True)

# Train the model
  = 50
lr = 0.0002
beta1 = 0.5
# Hyperparameters
nz = 100  # Size of z latent vector (input to generator)
ngf = 64  # Size of feature maps in generator
ndf = 64  # Size of feature maps in discriminator
nc = 3  # Number of color channels (RGB)
image_size = 256
batch_size = 32  # Set your batch size here
# Initialize models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
netG, netD = initialize_models(nz, ngf, ndf, nc, device)
train_dcgan_single_class_from_tensor(netG, netD, real_images, nz, num_epochs, lr, beta1, device, save_dir, batch_size)


Training Epochs:   2%|▏         | 1/50 [12:26<10:09:27, 746.28s/it]

Saved generator output at epoch 0 to gan_training_results_single_class\epoch_0_fake_samples.png
Epoch [0/50] Loss_D: 0.0013 Loss_G: 8.4893


Training Epochs:   6%|▌         | 3/50 [37:11<9:42:18, 743.37s/it] 

Saved generator output at epoch 2 to gan_training_results_single_class\epoch_2_fake_samples.png


Training Epochs:  10%|█         | 5/50 [1:01:58<9:17:41, 743.59s/it]

...

Saved generator output at epoch 38 to gan_training_results_single_class\epoch_38_fake_samples.png


Training Epochs:  82%|████████▏ | 41/50 [8:29:43<1:51:31, 743.54s/it]

Saved generator output at epoch 40 to gan_training_results_single_class\epoch_40_fake_samples.png
Epoch [40/50] Loss_D: 0.0000 Loss_G: 17.3720


Training Epochs:  84%|████████▍ | 42/50 [8:43:42<1:39:45, 748.16s/it]


KeyboardInterrupt: 

In [ ]:
# Create folder to save results
save_dir = "gan_training_results_single_class"
os.makedirs(save_dir, exist_ok=True)

# Train the model
num_epochs = 50
lr = 0.0002
beta1 = 0.5
# Hyperparameters
nz = 100  # Size of z latent vector (input to generator)
ngf = 64  # Size of feature maps in generator
ndf = 64  # Size of feature maps in discriminator
nc = 3  # Number of color channels (RGB)
image_size = 256
# Initialize models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
netG, netD = initialize_models(nz, ngf, ndf, nc, device)
train_dcgan_single_class_from_tensor(netG, netD, real_images, nz, num_epochs, lr, beta1, device, save_dir)

In [20]:




# Create folder to save results
save_dir = "gan_training_results_single_class"
os.makedirs(save_dir, exist_ok=True)

# Train the model using the real_images tensor
train_dcgan_single_class_from_tensor(netG, netD, real_images, nz, num_epochs, lr, beta1, device, save_dir)


ValueError: Using a target size (torch.Size([169])) that is different to the input size (torch.Size([1])) is deprecated. Please ensure they have the same size.